In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")
spark.sql("USE gold")


DataFrame[]

In [0]:
# === 1º PROJETO — ÁREA DE LOGÍSTICA (VENDAS POR LOCALIDADE) ===

print("=== Tabela gold_ft_vendas_consumidor_local ===")
spark.sql("SELECT * FROM gold.gold_ft_vendas_consumidor_local").show()

spark.sql("""
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor
AS
SELECT 
    cidade,
    estado,
    COUNT(*) as quantidade_vendas,
    SUM(valor_total_pedido_brl) as valor_total_localidade
FROM gold.gold_ft_vendas_consumidor_local
GROUP BY cidade, estado
""")

print("\n=== Amostra da view view_total_compras_por_consumidor ===")
spark.sql("SELECT * FROM gold.view_total_compras_por_consumidor ORDER BY valor_total_localidade DESC").show()

# Respondendo a pergunta da área de negócio: total de vendas por estado
print("\n=== Total de Vendas por Estado (Ordenado por maior valor) ===")
spark.sql("""
SELECT 
    estado,
    SUM(quantidade_vendas) as total_vendas,
    ROUND(SUM(valor_total_localidade), 2) as valor_total_estado
FROM gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY valor_total_estado DESC
""").show()

=== Tabela gold_ft_vendas_consumidor_local ===
+--------------------+--------------------+----------------------+--------------------+------+-----------+
|           id_pedido|       id_consumidor|valor_total_pedido_brl|              cidade|estado|data_pedido|
+--------------------+--------------------+----------------------+--------------------+------+-----------+
|09b70d085a1bfb3b4...|c1763055037cbee9f...|                  54.0|             Jundiai|    SP| 2017-05-20|
|6ea71515224447ac7...|3328102dd0972c652...|                  50.1|      Patos De Minas|    MG| 2017-07-21|
|9980c5519e01cc5a1...|68fcf3339b654cd98...|    216.60000000000002|              Mococa|    SP| 2017-01-17|
|1384e3bb7bc8ba5a4...|551f0a2f444b08c32...|                 74.32|Itapecerica Da Serra|    SP| 2017-08-07|
|5f2ca185b50a74d71...|e608c716333c772e7...|                 79.38|       Caxias Do Sul|    RS| 2018-06-06|
|0b768de519ebcf7df...|20aac2061bcd04913...|                667.71|           Apucarana|    PR| 20

In [0]:
# === 2º PROJETO — ÁREA DE LOGÍSTICA (ANÁLISE DE ATRASOS DE ENTREGAS) ===

print("=== Tabela gold.ft_atrasos_pedidos_local_vendedor ===")
spark.sql("SELECT * FROM gold.ft_atrasos_pedidos_local_vendedor").show()

spark.sql("""
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade
AS
SELECT 
    cidade,
    estado,
    CAST(ROUND(AVG(tempo_entrega_dias)) AS INT) as tempo_medio_entrega,
    CAST(ROUND(AVG(tempo_entrega_estimado_dias)) AS INT) as tempo_medio_estimado,
    CASE 
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
        ELSE 'NÃO'
    END as entrega_maior_que_estimado
FROM gold.ft_atrasos_pedidos_local_vendedor
WHERE entrega_no_prazo != 'Não Entregue'
GROUP BY cidade, estado
""")

print("\n=== Amostra da view view_tempo_medio_entrega_localidade ===")
spark.sql("SELECT * FROM gold.view_tempo_medio_entrega_localidade ORDER BY tempo_medio_entrega DESC LIMIT 10").show()

spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade
AS
SELECT 
    id_vendedor,
    COUNT(*) as total_pedidos,
    SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) as total_atrasados,
    ROUND((SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) / COUNT(*) * 100), 2) as percentual_atraso
FROM gold.ft_atrasos_pedidos_local_vendedor
WHERE entrega_no_prazo != 'Não Entregue'
GROUP BY id_vendedor
HAVING COUNT(*)
""")

print("\n=== Amostra da view view_vendedor_pontualidade (ordenado por maior percentual de atraso) ===")
spark.sql("SELECT * FROM gold.view_vendedor_pontualidade ORDER BY percentual_atraso DESC").show()

=== Tabela gold.ft_atrasos_pedidos_local_vendedor ===
+--------------------+--------------------+--------------------+----------------+------------------+---------------------------+----------------+------+
|           id_pedido|         id_vendedor|       id_consumidor|entrega_no_prazo|tempo_entrega_dias|tempo_entrega_estimado_dias|          cidade|estado|
+--------------------+--------------------+--------------------+----------------+------------------+---------------------------+----------------+------+
|ccbabeb0b02433bd0...|16090f2ca825584b5...|c77ee2d8ba1614a4d...|             Sim|                18|                         22|         Uberaba|    MG|
|c6bf92017bd40729c...|36a968b544695394e...|3d3c463710ea6e8dd...|             Sim|                 7|                          8|          Sumare|    SP|
|ab87dc5a5f1856a10...|b410bdd36d5db7a65...|538a4d02876412846...|             Sim|                 7|                         43|         Uberaba|    MG|
|06ff862a85c2402aa...|7e3f87

In [0]:
# === 3º PROJETO — ÁREA COMERCIAL (ANÁLISES DE VENDAS POR PERÍODO) ===

# 3.1 - Criar Dimensão de Tempo — gold.dm_tempo
spark.sql("""
CREATE OR REPLACE TABLE gold.dm_tempo
USING delta
AS
WITH date_sequence AS (
  SELECT explode(sequence(
    to_date('2016-01-01'), 
    to_date('2018-12-31'), 
    interval 1 day
  )) as sk_tempo
)
SELECT 
  sk_tempo,
  YEAR(sk_tempo) as ano,
  QUARTER(sk_tempo) as trimestre,
  MONTH(sk_tempo) as mes,
  WEEKOFYEAR(sk_tempo) as semana_do_ano,
  DAY(sk_tempo) as dia,
  DAYOFWEEK(sk_tempo) as dia_da_semana_num,
  CASE DAYOFWEEK(sk_tempo)
    WHEN 1 THEN 'Domingo'
    WHEN 2 THEN 'Segunda-feira'
    WHEN 3 THEN 'Terça-feira'
    WHEN 4 THEN 'Quarta-feira'
    WHEN 5 THEN 'Quinta-feira'
    WHEN 6 THEN 'Sexta-feira'
    WHEN 7 THEN 'Sábado'
  END as dia_da_semana_nome,
  CASE MONTH(sk_tempo)
    WHEN 1 THEN 'Janeiro'
    WHEN 2 THEN 'Fevereiro'
    WHEN 3 THEN 'Março'
    WHEN 4 THEN 'Abril'
    WHEN 5 THEN 'Maio'
    WHEN 6 THEN 'Junho'
    WHEN 7 THEN 'Julho'
    WHEN 8 THEN 'Agosto'
    WHEN 9 THEN 'Setembro'
    WHEN 10 THEN 'Outubro'
    WHEN 11 THEN 'Novembro'
    WHEN 12 THEN 'Dezembro'
  END as mes_nome,
  CASE 
    WHEN DAYOFWEEK(sk_tempo) IN (1, 7) THEN 'Sim'
    ELSE 'Não'
  END as eh_fim_de_semana
FROM date_sequence
""")

print("\n=== Amostra DISTRIBUÍDA da gold.dm_tempo(dos 3 anos) ===")
spark.sql("""
SELECT * FROM gold.dm_tempo 
WHERE 
    sk_tempo = '2016-01-01' OR
    sk_tempo = '2016-06-15' OR
    sk_tempo = '2016-12-31' OR
    sk_tempo = '2017-04-01' OR
    sk_tempo = '2017-08-15' OR
    sk_tempo = '2017-12-25' OR
    sk_tempo = '2018-03-20' OR
    sk_tempo = '2018-07-04' OR
    sk_tempo = '2018-09-15' OR
    sk_tempo = '2018-12-31'
ORDER BY sk_tempo
""").show(truncate=False)


=== Amostra DISTRIBUÍDA da gold.dm_tempo(dos 3 anos) ===
+----------+----+---------+---+-------------+---+-----------------+------------------+--------+----------------+
|sk_tempo  |ano |trimestre|mes|semana_do_ano|dia|dia_da_semana_num|dia_da_semana_nome|mes_nome|eh_fim_de_semana|
+----------+----+---------+---+-------------+---+-----------------+------------------+--------+----------------+
|2016-01-01|2016|1        |1  |53           |1  |6                |Sexta-feira       |Janeiro |Não             |
|2016-06-15|2016|2        |6  |24           |15 |4                |Quarta-feira      |Junho   |Não             |
|2016-12-31|2016|4        |12 |52           |31 |7                |Sábado            |Dezembro|Sim             |
|2017-04-01|2017|2        |4  |13           |1  |7                |Sábado            |Abril   |Sim             |
|2017-08-15|2017|3        |8  |33           |15 |3                |Terça-feira       |Agosto  |Não             |
|2017-12-25|2017|4        |12 |52     

In [0]:
# 3.2 - Criar a Tabela Fato gold.ft_vendas_geral
spark.sql("""
CREATE OR REPLACE TABLE gold.ft_vendas_geral
USING delta
AS
SELECT 
    i.id_pedido,
    CONCAT(i.id_pedido, '_', i.item_pedido) as id_item,
    p.id_cliente as fk_cliente,
    i.id_produto as fk_produto,
    i.id_vendedor as fk_vendedor,
    CAST(p.data_compra AS DATE) as fk_tempo,
    p.status_pedido,
    DATEDIFF(p.data_entrega_cliente, p.data_compra) as tempo_entrega_dias,
    CASE 
        WHEN p.data_entrega_cliente IS NULL THEN 'Não Entregue'
        WHEN p.data_entrega_cliente <= p.data_estimada_entrega THEN 'Sim'
        ELSE 'Não'
    END as entrega_no_prazo,
    i.preco as valor_produto_brl,
    i.valor_frete as valor_frete_brl,
    (i.preco + i.valor_frete) as valor_total_item_brl,
    ROUND(i.preco * 0.19, 2) as valor_produto_usd,
    ROUND(i.valor_frete * 0.19, 2) as valor_frete_usd,
    ROUND((i.preco + i.valor_frete) * 0.19, 2) as valor_total_item_usd,
    0.19 as cotacao_dolar,
    COALESCE(a.media_avaliacao, 0) as avaliacao_pedido
FROM silver.ft_item_pedido i
JOIN silver.dim_pedido p ON i.id_pedido = p.id_pedido
LEFT JOIN (
    SELECT id_pedido, AVG(pontuacao) as media_avaliacao
    FROM silver.ft_avaliacao_pedido 
    GROUP BY id_pedido
) a ON i.id_pedido = a.id_pedido
WHERE p.data_compra IS NOT NULL
""")

print("\n=== Amostra da gold.ft_vendas_geral ===")
spark.sql("SELECT * FROM gold.ft_vendas_geral").show(truncate=False)


=== Amostra da gold.ft_vendas_geral ===
+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+----------+-------------+------------------+----------------+-----------------+---------------+--------------------+-----------------+---------------+--------------------+-------------+----------------+
|id_pedido                       |id_item                           |fk_cliente                      |fk_produto                      |fk_vendedor                     |fk_tempo  |status_pedido|tempo_entrega_dias|entrega_no_prazo|valor_produto_brl|valor_frete_brl|valor_total_item_brl|valor_produto_usd|valor_frete_usd|valor_total_item_usd|cotacao_dolar|avaliacao_pedido|
+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+----------+-------------+------------------+-----

In [0]:
# === 3.3 - VIEW VENDAS POR PERÍODO E CONSULTAS ANALÍTICAS ===

spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendas_por_periodo
AS
SELECT 
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,
    COUNT(DISTINCT v.id_pedido) as total_pedidos,
    COUNT(v.id_item) as total_itens,
    ROUND(SUM(v.valor_total_item_brl), 2) as receita_total_brl,
    ROUND(SUM(v.valor_total_item_usd), 2) as receita_total_usd,
    ROUND(AVG(v.valor_total_item_brl), 2) as ticket_medio_brl,
    ROUND(AVG(CASE WHEN v.avaliacao_pedido > 0 THEN v.avaliacao_pedido END), 2) as avaliacao_media
FROM gold.ft_vendas_geral v
JOIN gold.dm_tempo t ON v.fk_tempo = t.sk_tempo
GROUP BY 
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num
""")

print("\n=== Amostra da view view_vendas_por_periodo ===")
spark.sql("SELECT * FROM gold.view_vendas_por_periodo ORDER BY receita_total_brl DESC").show()

print("\n=== CONSULTAS ANALÍTICAS ===")

print("1. Qual é o dia da semana com maior receita total em reais?")
spark.sql("""
SELECT 
    dia_da_semana_num,
    CASE dia_da_semana_num
        WHEN 1 THEN 'Domingo'
        WHEN 2 THEN 'Segunda-feira'
        WHEN 3 THEN 'Terça-feira'
        WHEN 4 THEN 'Quarta-feira'
        WHEN 5 THEN 'Quinta-feira'
        WHEN 6 THEN 'Sexta-feira'
        WHEN 7 THEN 'Sábado'
    END as dia_semana,
    SUM(receita_total_brl) as receita_total
FROM gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total DESC
LIMIT 1
""").show()

print("2. Considerando o último ano disponível, qual foi o mês com maior ticket médio?")
spark.sql("""
SELECT 
    mes,
    mes_nome,
    ROUND(AVG(ticket_medio_brl), 2) as ticket_medio
FROM gold.view_vendas_por_periodo
WHERE ano = (SELECT MAX(ano) FROM gold.view_vendas_por_periodo)
GROUP BY mes, mes_nome
ORDER BY ticket_medio DESC
LIMIT 1
""").show()


=== Amostra da view view_vendas_por_periodo ===
+----+---------+---+---------+---+-----------------+-------------+-----------+-----------------+-----------------+----------------+---------------+
| ano|trimestre|mes| mes_nome|dia|dia_da_semana_num|total_pedidos|total_itens|receita_total_brl|receita_total_usd|ticket_medio_brl|avaliacao_media|
+----+---------+---+---------+---+-----------------+-------------+-----------+-----------------+-----------------+----------------+---------------+
|2017|        4| 11| Novembro| 24|                6|         1166|       1366|        178377.63|         33892.02|          130.58|           3.64|
|2017|        4| 11| Novembro| 25|                7|          498|        580|         71858.16|         13653.14|          123.89|           3.58|
|2018|        3|  8|   Agosto|  6|                2|          372|        430|         65860.42|         12513.55|          153.16|           4.17|
|2018|        2|  5|     Maio| 16|                4|          3

In [0]:
# === 3.4 - VIEW TOP PRODUTOS ===

spark.sql("""
CREATE OR REPLACE VIEW gold.view_top_produto
AS
SELECT 
    v.fk_produto as id_produto,
    p.categoria_produto,
    COUNT(v.id_item) as quantidade_vendida,
    COUNT(DISTINCT v.id_pedido) as total_pedidos,
    ROUND(SUM(v.valor_total_item_brl), 2) as receita_brl,
    ROUND(SUM(v.valor_total_item_usd), 2) as receita_usd,
    ROUND(AVG(v.valor_produto_brl), 2) as preco_medio_brl,
    ROUND(AVG(CASE WHEN v.avaliacao_pedido > 0 THEN v.avaliacao_pedido END), 2) as avaliacao_media,
    ROUND(AVG(p.peso_g), 2) as peso_medio_gramas
FROM gold.ft_vendas_geral v
JOIN silver.dim_produto p ON v.fk_produto = p.id_produto
GROUP BY v.fk_produto, p.categoria_produto
""")

print("\n=== Amostra da view view_top_produto (ordenado por maior receita) ===")
spark.sql("SELECT * FROM gold.view_top_produto ORDER BY receita_brl DESC").show()

print("\n=== Top categorias por receita ===")
spark.sql("""
SELECT 
    categoria_produto,
    COUNT(*) as total_produtos,
    SUM(quantidade_vendida) as total_vendido,
    ROUND(SUM(receita_brl), 2) as receita_total_brl
FROM gold.view_top_produto
GROUP BY categoria_produto
ORDER BY receita_total_brl DESC
LIMIT 10
""").show()


=== Amostra da view view_top_produto (ordenado por maior receita) ===
+--------------------+--------------------+------------------+-------------+-----------+-----------+---------------+---------------+-----------------+
|          id_produto|   categoria_produto|quantidade_vendida|total_pedidos|receita_brl|receita_usd|preco_medio_brl|avaliacao_media|peso_medio_gramas|
+--------------------+--------------------+------------------+-------------+-----------+-----------+---------------+---------------+-----------------+
|bb50f2e236e5eea01...|        Beleza_saude|               195|          187|    67606.1|   12844.96|         327.62|           4.22|            400.0|
|d1c427060a0f73f6b...|Informatica_acess...|               343|          323|   60976.03|   11585.25|         137.65|           4.19|           6550.0|
|6cdd53843498f9289...|        Beleza_saude|               156|          151|   59093.99|   11227.81|         350.83|           4.31|            900.0|
|99a4788cb24856965...| 

In [0]:
# === 3.5 - VIEW VENDAS PRODUTOS ESTÉTICOS (FASHION) - ===

spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos
AS
WITH vendas_fashion AS (
    SELECT 
        v.id_item,
        v.id_pedido,
        v.fk_produto,
        v.fk_tempo,
        v.valor_total_item_brl,
        v.valor_total_item_usd,
        v.avaliacao_pedido,
        p.categoria_produto
    FROM gold.ft_vendas_geral v
    JOIN silver.dim_produto p ON v.fk_produto = p.id_produto
    WHERE p.categoria_produto LIKE 'Fashion%' 
       OR p.categoria_produto = 'Beleza_saude'
       OR p.categoria_produto = 'Perfumaria'
       OR p.categoria_produto = 'Malas_acessorios'
),
vendas_agrupadas AS (
    SELECT 
        t.ano,
        t.mes,
        vf.categoria_produto,
        COUNT(DISTINCT vf.id_pedido) as total_pedidos,
        COUNT(vf.id_item) as total_itens_vendidos,
        ROUND(SUM(vf.valor_total_item_brl), 2) as receita_total_brl,
        ROUND(SUM(vf.valor_total_item_usd), 2) as receita_total_usd,
        ROUND(AVG(vf.valor_total_item_brl), 2) as ticket_medio_brl,
        ROUND(AVG(vf.valor_total_item_usd), 2) as ticket_medio_usd,
        ROUND(AVG(CASE WHEN vf.avaliacao_pedido > 0 THEN vf.avaliacao_pedido END), 2) as avaliacao_media
    FROM vendas_fashion vf
    JOIN gold.dm_tempo t ON vf.fk_tempo = t.sk_tempo
    GROUP BY t.ano, t.mes, vf.categoria_produto
)
SELECT * FROM vendas_agrupadas
""")

print("\n=== Amostra da view view_vendas_produtos_esteticos ===")
spark.sql("SELECT * FROM gold.view_vendas_produtos_esteticos ORDER BY receita_total_brl DESC").show()

print("\n=== Estatísticas gerais por categoria fashion ===")
spark.sql("""
SELECT 
    categoria_produto,
    SUM(total_pedidos) as total_pedidos,
    SUM(total_itens_vendidos) as total_itens,
    ROUND(SUM(receita_total_brl), 2) as receita_total_brl,
    ROUND(AVG(ticket_medio_brl), 2) as ticket_medio_brl,
    ROUND(AVG(avaliacao_media), 2) as avaliacao_media
FROM gold.view_vendas_produtos_esteticos
GROUP BY categoria_produto
ORDER BY receita_total_brl DESC
""").show()

=== View gold.view_vendas_produtos_esteticos criada com sucesso! ===

=== Amostra da view view_vendas_produtos_esteticos ===
+----+---+-----------------+-------------+--------------------+-----------------+-----------------+----------------+----------------+---------------+
| ano|mes|categoria_produto|total_pedidos|total_itens_vendidos|receita_total_brl|receita_total_usd|ticket_medio_brl|ticket_medio_usd|avaliacao_media|
+----+---+-----------------+-------------+--------------------+-----------------+-----------------+----------------+----------------+---------------+
|2018|  8|     Beleza_saude|          778|                 849|        137206.07|         26069.53|          161.61|           30.71|           4.36|
|2018|  6|     Beleza_saude|          799|                 885|        125197.75|         23787.74|          141.47|           26.88|           4.36|
|2018|  7|     Beleza_saude|          713|                 784|        122743.98|         23321.47|          156.56|         